In [1]:
import numpy as np
from submit import my_kernel
from submit import my_decode
import time as tm
from sklearn.kernel_ridge import KernelRidge

In [4]:
n_train = 4000
n_test = 1000
x_train = np.loadtxt( "src/dummy/secret_x_trn.txt" ).reshape( ( n_train, 1 ) )
x_test = np.loadtxt( "src/dummy/secret_x_tst.txt" ).reshape( ( n_test, 1 ) )
Z_train = np.loadtxt( "src/dummy/secret_Z_trn.txt" )
Z_test = np.loadtxt( "src/dummy/secret_Z_tst.txt" )
y_train = np.loadtxt( "src/dummy/secret_y_trn.txt" ).reshape( ( n_train, 1 ) )
y_test = np.loadtxt( "src/dummy/secret_y_tst.txt" ).reshape( ( n_test, 1 ) )

n_trials = 5

t_kernel = 0
t_train = 0
R_score = 0

In [5]:
rgs = KernelRidge( kernel = "precomputed" )
for t in range( n_trials ):
  tic = tm.perf_counter()
  G_train = my_kernel( x_train, Z_train, x_train, Z_train )
  G_test = my_kernel( x_test, Z_test, x_train, Z_train )
  toc = tm.perf_counter()

  t_kernel += toc - tic

  tic = tm.perf_counter()
  rgs.fit( G_train, y_train )
  toc = tm.perf_counter()
  t_train += toc - tic

  R_score += rgs.score( G_test, y_test )

In [7]:
t_kernel /= n_trials
t_train /= n_trials
R_score /= n_trials

In [8]:
def get_APUF_model( p, q, r, s ):
  p = np.maximum( p, 0 )
  q = np.maximum( q, 0 )
  r = np.maximum( r, 0 )
  s = np.maximum( s, 0 )
  d = p - q
  c = r - s
  alpha = ( d + c ) / 2
  beta = ( d - c ) / 2
  w = np.zeros( ( len( alpha ) + 1, )  )
  w[:-1] += alpha
  w[1:] += beta
  return w

def get_XOR_APUF_model( a, b, c, d, p, q, r, s ):
  return np.kron( get_APUF_model( a, b, c, d ), get_APUF_model( p, q, r, s ) )

In [9]:
t_decode = 0
m_dist = 0

In [11]:
W = np.loadtxt( "src/dummy/secret_mod.txt" )
( n_models, dims ) = W.shape
for t in range( n_trials ):
  for itr in range( n_models ):
    w = W[ itr, : ]
    tic = tm.perf_counter()
    a_hat, b_hat, c_hat, d_hat, p_hat, q_hat, r_hat, s_hat = my_decode( w )
    toc = tm.perf_counter()
    t_decode += toc - tic
    w_hat = get_XOR_APUF_model( a_hat, b_hat, c_hat, d_hat, p_hat, q_hat, r_hat, s_hat )
    m_dist += np.linalg.norm( w - w_hat )

In [12]:
t_decode /= ( n_trials * n_models )
m_dist /= ( n_trials * n_models )

In [13]:
print( f"{t_kernel},{t_train},{R_score},{t_decode},{m_dist}" )

0.042148071359952154,0.09926077604002785,0.1939891689644019,0.0002272084599826485,4.3410422447980044e-16
